In [1]:
#Imports
import pandas as pd
import geopandas
import numpy as np
from shapely.geometry import Point

In [2]:
#Get both dataframes with the information of bicimad stations and place of interest
df_bicimad = pd.read_csv('..\data\clean_bicimad.csv')
df_embajadas = pd.read_csv('..\data\clean_embajadas.csv')

In [3]:
#Check if the bicimad dataframe is correct
df_bicimad[:3]

,BiciMAD station,Station location,lat_bici,long_bici
0,1a - Puerta del Sol A,Puerta del Sol nº 1,40.417214,-3.701834
1,1b - Puerta del Sol B,Puerta del Sol nº 1,40.417313,-3.701603
2,2 - Miguel Moya,Calle Miguel Moya nº 1,40.420589,-3.705842


In [4]:
#Check if the place of interest dataframe is correct
df_embajadas[:3]

,Place of interest,Type of place,address,embajada_latitude,embajadas_longitude
0,Consulado de Argentina,Consulado,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666
1,Consulado de Belice,Consulado,CALLE TALAVERA 9,40.453376,-3.680978
2,Consulado de Bolivia,Consulado,CALLE AVIADOR LINDBERGH 3,40.442447,-3.679665


In [ ]:
#merged_df = pd.merge(df_bicimad, df_embajadas, left_index=True, right_index=True)
#merged_df[:3]
#merged_df.info()

In [5]:
#Run the function
#transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
def to_mercator(lat, long):
    c = geopandas.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [6]:
#Run the function
# return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [7]:
%%time

distance_total = []
for idx, rows_e in df_embajadas.iterrows():
    for idx, rows_b in df_bicimad.iterrows():
        distance_total.append(distance_meters(rows_e['embajada_latitude'],rows_e['embajadas_longitude'],rows_b['lat_bici'],rows_b['long_bici']))
print(distance_total)
len(distance_total)

[0    1595.648921
dtype: float64, 0    1570.712523
dtype: float64, 0    1697.402412
dtype: float64, 0    1606.478853
dtype: float64, 0    1107.400361
dtype: float64, 0    1047.354555
dtype: float64, 0    788.019491
dtype: float64, 0    661.673359
dtype: float64, 0    311.900114
dtype: float64, 0    2345.334508
dtype: float64, 0    326.608083
dtype: float64, 0    1226.945818
dtype: float64, 0    1509.160095
dtype: float64, 0    1985.398844
dtype: float64, 0    2333.717152
dtype: float64, 0    1663.306468
dtype: float64, 0    1751.738011
dtype: float64, 0    1362.015218
dtype: float64, 0    1136.993967
dtype: float64, 0    774.749276
dtype: float64, 0    1047.379546
dtype: float64, 0    1793.086779
dtype: float64, 0    1937.407121
dtype: float64, 0    2315.229195
dtype: float64, 0    1896.710869
dtype: float64, 0    1955.573223
dtype: float64, 0    525.76338
dtype: float64, 0    1695.009959
dtype: float64, 0    1292.145351
dtype: float64, 0    1374.099048
dtype: float64, 0    2328.36108


41712

In [8]:
distance_total = np.array(distance_total).reshape(len(df_embajadas), len(df_bicimad))


In [9]:
min_indexes = np.argmin(distance_total, axis=1)
df_embajadas['min_distance'] = np.min(distance_total, axis=1)
df_embajadas['closest_station'] = df_bicimad.iloc[min_indexes]['BiciMAD station'].reset_index(drop=True)

In [10]:
df_embajadas[:3]

,Place of interest,Type of place,address,embajada_latitude,embajadas_longitude,min_distance,closest_station
0,Consulado de Argentina,Consulado,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,40.428216,-3.692666,311.900114,8 - Alonso Martínez
1,Consulado de Belice,Consulado,CALLE TALAVERA 9,40.453376,-3.680978,277.816571,148 - Serrano 210
2,Consulado de Bolivia,Consulado,CALLE AVIADOR LINDBERGH 3,40.442447,-3.679665,224.503569,146 - María Francisca 1


In [11]:
df_embajadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Place of interest    158 non-null    object 
 1   Type of place        158 non-null    object 
 2   address              158 non-null    object 
 3   embajada_latitude    157 non-null    float64
 4   embajadas_longitude  157 non-null    float64
 5   min_distance         157 non-null    float64
 6   closest_station      158 non-null    object 
dtypes: float64(3), object(4)
memory usage: 8.8+ KB


In [12]:
df_bicimad_embajadas = df_embajadas[["Place of interest","Type of place","address","closest_station"]]
df_bicimad_embajadas[:5]

,Place of interest,Type of place,address,closest_station
0,Consulado de Argentina,Consulado,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,8 - Alonso Martínez
1,Consulado de Belice,Consulado,CALLE TALAVERA 9,148 - Serrano 210
2,Consulado de Bolivia,Consulado,CALLE AVIADOR LINDBERGH 3,146 - María Francisca 1
3,Consulado de Brasil,Consulado,CALLE GOYA 5 y 7 pasaje,93 - Plaza de Colón
4,Consulado de Bélgica,Consulado,PASEO CASTELLANA 18 PLANTA 6 Izquierda,104 - Castellana


In [19]:
df_bicimad_embajadas.rename(columns = {'closest_station':'BiciMAD station'}, inplace = True)
df_bicimad_embajadas

C:\Users\lourdes.ruiz\AppData\Local\Temp\ipykernel_19100\3915771642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bicimad_embajadas.rename(columns = {'closest_station':'BiciMAD station'}, inplace = True)


,Place of interest,Type of place,address,BiciMAD station
0,Consulado de Argentina,Consulado,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,8 - Alonso Martínez
1,Consulado de Belice,Consulado,CALLE TALAVERA 9,148 - Serrano 210
2,Consulado de Bolivia,Consulado,CALLE AVIADOR LINDBERGH 3,146 - María Francisca 1
3,Consulado de Brasil,Consulado,CALLE GOYA 5 y 7 pasaje,93 - Plaza de Colón
4,Consulado de Bélgica,Consulado,PASEO CASTELLANA 18 PLANTA 6 Izquierda,104 - Castellana
...,...,...,...,...
153,Embajada de Vietnam,Embajada,AVENIDA ALFONSO XIII 54,251 - Corazón de María
154,Embajada de Yemen,Embajada,PASEO CASTELLANA 117 PLANTA 8 D,156 - Sor Ángela de la Cruz
155,Embajada de la República Popular Democrática d...,Embajada,CALLE DARIO APARICIO 43,261 - Facultad Derecho
156,Embajada de la República de Corea,Embajada,CALLE GONZALEZ AMIGO 15,254 - Pío XII


In [15]:
df_bicimad_stations = df_bicimad[["BiciMAD station","Station location"]]
df_bicimad_stations[:3]

,BiciMAD station,Station location
0,1a - Puerta del Sol A,Puerta del Sol nº 1
1,1b - Puerta del Sol B,Puerta del Sol nº 1
2,2 - Miguel Moya,Calle Miguel Moya nº 1


In [25]:
final_df = pd.merge(df_bicimad_embajadas, df_bicimad_stations, on='BiciMAD station')
final_df[:3]

,Place of interest,Type of place,address,BiciMAD station,Station location
0,Consulado de Argentina,Consulado,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5
1,Embajada de Argentina,Embajada,CALLE FERNANDO EL SANTO 15 PLANTA 1,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5
2,Embajada de Brasil,Embajada,CALLE FERNANDO EL SANTO 6,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5


In [33]:
final_df.rename(columns = {'address':'Place address'}, inplace = True)
final_df[:15]

,Place of interest,Type of place,Place address,BiciMAD station,Station location
0,Consulado de Argentina,Consulado,CALLE FERNANDO EL SANTO 15 PLANTA BAJA,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5
1,Embajada de Argentina,Embajada,CALLE FERNANDO EL SANTO 15 PLANTA 1,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5
2,Embajada de Brasil,Embajada,CALLE FERNANDO EL SANTO 6,8 - Alonso Martínez,Plaza de Alonso Martínez nº 5
3,Consulado de Belice,Consulado,CALLE TALAVERA 9,148 - Serrano 210,Calle Serrano nº 210
4,Embajada de Kenia,Embajada,CALLE SEGRE 13,148 - Serrano 210,Calle Serrano nº 210
5,Embajada de Lituania,Embajada,CALLE PISUERGA 5,148 - Serrano 210,Calle Serrano nº 210
6,Embajada de Marruecos,Embajada,CALLE SERRANO 179,148 - Serrano 210,Calle Serrano nº 210
7,Embajada de Nigeria,Embajada,CALLE SEGRE 23,148 - Serrano 210,Calle Serrano nº 210
8,Consulado de Bolivia,Consulado,CALLE AVIADOR LINDBERGH 3,146 - María Francisca 1,Calle María Francisca nº 1
9,Embajada de San Marino,Embajada,CALLE PADRE JESUS ORDO&amp;Ntilde;EZ 18 PLANTA...,146 - María Francisca 1,Calle María Francisca nº 1


In [27]:
final_df.to_csv(r'..\data\final_result.csv', index=False)